In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings
import math

import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

warnings.filterwarnings('ignore')


# **Variation Coefficient Ratio Analysis**

In [3]:
def save_dict_list_as(dic, file_name):
    with open(file_name, 'wb') as file:
        pickle.dump(dic, file)
    return 0
    

def load_dict_list(file_name):
    with open(file_name, 'rb') as file:
        loaded_dict = pickle.load(file)
    return loaded_dict

In [4]:
data = pd.read_csv("data_set/data_for_analysis.csv")  # Load the prepared data for the analysis
features = load_dict_list("Food_group/features.pkl")
numb = len(data['subject_key'].unique())
print(f"Number of subjects: {numb}")


Number of subjects: 453


**Compute variance using Linear Mixed Models**

In [5]:
res = {}
for f in tqdm(features) : 
    modele_comp = smf.mixedlm(f"{f} ~ C(week_day) + C(gender) + C(bmi_cat) + C(age_group)", data, groups=data["subject_key"])
    result = modele_comp.fit()
    subject = pd.DataFrame()
    subject["subject_key"] = data["subject_key"]
    subject["residual"] = result.resid
    var = subject.groupby('subject_key')['residual'].var()
    mean_within_person_variance = var.mean()
    between_person_variance = result.cov_re.iloc[0,0]
    mean = data[f].mean()
    median = data[f].median()
    std = data[f].std()
    CVw = np.sqrt(mean_within_person_variance) / mean * 100
    CVb = np.sqrt(between_person_variance) / mean * 100
    variance_ratio = (CVw ** 2) / (CVb ** 2)
    res[f] = {
        'mean' : mean,
        'median' : median,
        'CVw': CVw,
        'CVb': CVb,
        'variance_ratio': variance_ratio,
        'std': std,
    }
save_dict_list_as(res, "Var_Coef_Ratio_Results/variation_coef.pkl")

100%|██████████| 37/37 [02:01<00:00,  3.28s/it]


0

**Apply formula for differents r values**

In [6]:
def compute_number_of_days(r : list):
    res = load_dict_list("Var_Coef_Ratio_Results/variation_coef.pkl")
    d_values = {}
    for f in features: 
        d_values[f] = {}
        ratio = res[f]['variance_ratio']
        for r_val in r:
            d_r = (r_val**2/(1-r_val**2) * ratio)
            d_values[f][r_val] = math.ceil(d_r)
    return d_values

d_values = compute_number_of_days([0.8,0.85,0.9])


In [7]:
df = pd.DataFrame(columns= features)
for f in features:
    for metric, val in res[f].items():
        df.loc[metric,f] = val
d_val = pd.DataFrame(d_values)
df = pd.merge(df, d_val, how='outer')
index = list(res["protein_eaten"].keys()) + ["Days (r=0.8)", "Days (r=0.85)", "Days (r=0.9)"]
df.index = index
df = df.T


**create table**

In [8]:
table = df.copy()
def scientific_notation(value):
    if abs(value) < 1 and value != 0:
        return '{:.2e}'.format(value)
    else:
        return '{:.2f}'.format(float(value))    
mean = list(map(scientific_notation, df["mean"].tolist()))
std = list(map(scientific_notation, df["std"].tolist()))
mean_std = [f"{m} ± {s}" for m, s in zip(mean, std)]
table["mean"] = mean_std
table.drop("std", axis=1, inplace=True)
table.rename(columns={"mean": "Mean ± Std", "median": "Median", "CVw": "CVw (%)", "CVb": "CVb (%)", "variance_ratio": "Variance Ratio"}, inplace=True)
table = table.reset_index().rename(columns={'index': "Nutritional Feature"})
table.to_csv("Var_Coef_Ratio_Results/table_Var_coef_Ratio.csv", index=False)
table

,Nutritional Feature,Mean ± Std,Median,CVw (%),CVb (%),Variance Ratio,Days (r=0.8),Days (r=0.85),Days (r=0.9)
0,eaten_quantity_in_gram,3225.63 ± 1491.44,3063.0,25.60353,38.502801,0.442196,1,2,2
1,energy_kcal_eaten,2544.42 ± 661.79,2425.025,21.720033,11.601229,3.505201,7,10,15
2,energy_kj_eaten,10712.70 ± 2776.97,10249.6096,21.585064,11.609609,3.456778,7,10,15
3,carb_eaten,263.34 ± 80.52,254.15,24.473891,16.502227,2.199485,4,6,10
4,fat_eaten,109.52 ± 38.93,103.468,30.314855,16.489321,3.379913,7,9,15
5,protein_eaten,93.47 ± 33.05,88.663,29.71956,16.530885,3.232158,6,9,14
6,fiber_eaten,28.32 ± 12.83,26.2505,34.286507,29.017461,1.396136,3,4,6
7,alcohol_eaten,10.06 ± 17.95,0.0,148.79382,86.483659,2.960068,6,8,13
8,beta_carotene_eaten,3.31e-03 ± 3.84e-03,0.001851,101.445714,55.051225,3.395732,7,9,15
9,calcium_eaten,8.03e-01 ± 5.13e-01,0.696265,51.945532,36.566742,2.018012,4,6,9
